In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import time
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from scipy import stats
import statistics as stat


In [2]:
ds=pd.read_csv("TrainingDataset.arff.txt",header=None, comment='@')
#train, test = train_test_split(ds, test_size=0.2)

In [ ]:
start_time= time.time()

outer=KFold(n_splits=10)
outer.get_n_splits(ds)
C_range = np.logspace(-3 , 3, num=7)
parameters = {'C':C_range, 'kernel':['linear']}
results=dict()
count=0
for train_index, test_index in outer.split(ds):
    count+=1
    elapsed_time= time.time()
    print("\nIterazione",count)
    print("train+validation:", train_index, "test:", test_index)
    X_train, X_test= ds.iloc[train_index,:np.shape(ds)[1]-1], ds.iloc[test_index,:np.shape(ds)[1]-1]
    Y_train, Y_test= ds.iloc[train_index,-1], ds.iloc[test_index,-1]
    
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters, cv=10, scoring="accuracy",n_jobs=-1, return_train_score=True)
    clf.fit(X_train, Y_train)
    
    for i in range(0,len(clf.cv_results_['params'])):
        print ("Modello:",clf.cv_results_['params'][i], "accuracy:",clf.cv_results_['mean_test_score'][i])
    
    t_score= clf.score(X_test, Y_test)
    print ("Miglior modello:",clf.best_params_,"con score sul test set:",t_score)

    if str(clf.best_params_) in results:
        results[str(clf.best_params_)]['occurrency']+=1
        results[str(clf.best_params_)]['tests'].append(t_score)
    else:
        results[str(clf.best_params_)]=dict()
        results[str(clf.best_params_)]['occurrency']=1
        results[str(clf.best_params_)]['tests']=[]
        results[str(clf.best_params_)]['tests'].append(t_score)
        
    elapsed_time=time.time()-elapsed_time
    print("\nElapsed time:",elapsed_time)   

elapsed_time=time.time()-start_time
print("\nTotal elapsed time:",elapsed_time)        
    


Iterazione 1
train+validation: [ 1106  1107  1108 ... 11052 11053 11054] test: [   0    1    2 ... 1103 1104 1105]


In [28]:
print(results,"\n")
flag=False
duplicates=[]
tot=0
for x in results:
    tot=tot+results[x]['occurrency']
occ=0
best=None
for key in results:
    if results[key]['occurrency']>occ:
        occ=results[key]['occurrency']
        best=key
        testscores=results[key]['tests']
    elif results[key]['occurrency']==occ:
        flag=True
        duplicates.append(key)
print(best,"occorre",occ,"volte su",tot,".\nMedia dei test score:",np.mean(testscores))
if flag:
    print("Ci sono più modelli con la stessa occorrenza:")
    for x in duplicates:
        print(x,"occorre",results[x]['occurrency'],"volte su",tot,".\nMedia dei test score:",np.mean(results[x]['tests']))

{"{'C': 10.0, 'kernel': 'linear'}": {'occurrency': 9, 'tests': [0.9249547920433996, 0.9276672694394213, 0.9276672694394213, 0.9312839059674503, 0.9258589511754068, 0.9384615384615385, 0.9122171945701357, 0.92579185520362, 0.9294117647058824]}, "{'C': 0.1, 'kernel': 'linear'}": {'occurrency': 1, 'tests': [0.9276018099547512]}} 

{'C': 10.0, 'kernel': 'linear'} occorre 9 volte su 10 .
Media dei test score: 0.9270349490006973
